In [2]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [3]:
pd.options.display.max_rows = 50 

In [4]:
database_path = "/kaggle/input/largedata/Bool.db"

In [5]:
connection = sqlite3.connect(database_path)

In [23]:
tables = pd.read_sql("""

SELECT v.Ven_Phone_number AS Venue_Phone, up.phone_number AS User_Phone
FROM UserProfile up
JOIN VenueReview vr ON vr.Username = up.Username 
JOIN Venue v ON vr.Ven_Name = v.VenName 
JOIN ProductReview pr ON pr.Username = up.Username 
JOIN Product p ON p.ProductID = pr.ProductReviewID 
WHERE DATE(pr.PRCreatedAt) > DATE('2020-07-20') 
    AND p.price > 100
    AND vr.Rating > 2;



""", connection)

print(tables.head(10))

Empty DataFrame
Columns: [Venue_Phone, User_Phone]
Index: []


In [7]:
VenRating = pd.read_sql("""

Select avg(r.likes) as Rating, r.Ven_Name, v.Location from VenueReview r
    Inner Join Venue v on r.Ven_Name = v.VenName
    Group by r.Ven_Name
    Order by Rating;

""", connection)

print(VenRating.head(10))

   Rating          Ven_Name                                           Location
0     0.0      Abbott Group        003 Gates Oval\nSouth Anthonystad, ME 87021
1     0.0        Abbott Inc       00879 Brooks Fall\nWest Ashleyview, SC 19242
2     0.0        Abbott LLC   0032 Mccoy Street Apt. 544\nNguyentown, GU 65911
3     0.0        Abbott Ltd          004 Harmon Camp\nSouth Joymouth, IA 98905
4     0.0        Abbott PLC  0019 Jacob Streets Apt. 034\nSouth Mindyfort, ...
5     0.0   Abbott and Sons  0060 Peter Greens Suite 737\nDelacruzfort, MN ...
6     0.0      Abbott-Adams   19665 Blanchard Glens\nAndersonborough, PA 88289
7     0.0  Abbott-Alexander  5178 Alfred Tunnel Apt. 918\nSouth Megan, OR 8...
8     0.0      Abbott-Allen  27059 Valencia Terrace Apt. 442\nFranciscoport...
9     0.0   Abbott-Anderson  19836 Stacey Land Suite 391\nPort Jared, ND 79740


In [8]:
numF = pd.read_sql("""

SELECT u.Username, COUNT(DISTINCT f.TargetedUsername) AS "Num Friends"
    FROM UserProfile u
    INNER JOIN Friends f ON f.Username = u.Username
    GROUP BY u.Username
    HAVING COUNT(DISTINCT f.TargetedUsername) > 0
    ORDER BY "Num Friends"


""", connection)

print(numF.head(10))

     Username  Num Friends
0  0004zSqj69            1
1  000I0nkP22            1
2  000a2jMB93            1
3  000q8jPQ54            1
4  00100r5p03            1
5  001AxpAx62            1
6  001BPENc55            1
7  001M5MEF26            1
8  001Mupy499            1
9  001WAMbG88            1


In [9]:
bestV = pd.read_sql("""

Select v.VenName as "Venue Name", avg(r.likes) as Rating, v.Ven_Phone_number from Venue v
    Inner Join VenueReview r on r.Ven_Name = v.VenName
    Group By "Venue Name"
    Order by Rating;

""", connection)

print(bestV.head(10))

         Venue Name  Rating        Ven_Phone_number
0      Abbott Group     0.0      657.247.7935x95527
1        Abbott Inc     0.0   +1-704-218-0816x61731
2        Abbott LLC     0.0    +1-919-905-7245x3505
3        Abbott Ltd     0.0       (486)576-9573x916
4        Abbott PLC     0.0           (477)396-3582
5   Abbott and Sons     0.0   001-777-924-1043x2647
6      Abbott-Adams     0.0     +1-309-620-2446x961
7  Abbott-Alexander     0.0           (437)389-1744
8      Abbott-Allen     0.0  001-442-314-6170x33672
9   Abbott-Anderson     0.0            330.254.1334


In [10]:
friends = pd.read_sql("""
SELECT Status, StartTime  
FROM Friends
WHERE DATE(StartTime) > DATE('2020-07-20');

""", connection)

print(friends.head(10))


   Status   StartTime
0       2  2023-11-21
1       1  2021-03-06
2       2  2020-10-20
3       1  2021-02-08
4       0  2023-10-15
5       0  2020-09-19
6       1  2022-12-12
7       0  2022-02-10
8       0  2022-03-08
9       2  2022-01-29


In [12]:
postData = pd.read_sql("""
Select v.Category, vr.Rating, vr.WaitTimeReported from Venue v
    inner join VenueReview vr on v.VenName = vr.Ven_Name
    where vr.Rating > 2
    Group By v.VenName
    Order by vr.Rating;
       """, connection)

print(postData.head(10))

     Category  Rating WaitTimeReported
0         Bar       3  0 days 00:23:00
1         Bar       3  0 days 00:36:00
2        Park       3  0 days 00:50:00
3         Bar       3  0 days 00:10:00
4        Park       3  0 days 00:53:00
5       Hotel       3  0 days 00:58:00
6     Stadium       3  0 days 00:32:00
7  Restaurant       3  0 days 00:53:00
8  Restaurant       3  0 days 00:11:00
9       Hotel       3  0 days 00:55:00


In [14]:
RatingOn = pd.read_sql("""
SELECT Ven_Name, Rating 
FROM VenueReview  
WHERE DATE(VRCreatedAT) > DATE('1997-07-20') AND DATE(VRCreatedAT) < DATE('2015-07-20')
GROUP BY Ven_Name
ORDER BY Rating;


""", connection)

print(RatingOn.head(10))


                       Ven_Name  Rating
0                    Abbott LLC       1
1                    Abbott PLC       1
2     Abbott, Butler and Molina       1
3       Abbott, Dillon and Ford       1
4     Abbott, Gordon and Miller       1
5  Abbott, Harding and Williams       1
6  Abbott, Harrison and Vasquez       1
7        Abbott, Mccoy and West       1
8   Abbott, Mcdowell and Rogers       1
9  Abbott, Newman and Armstrong       1


In [15]:
avgRating = pd.read_sql("""
Select avg(r.Likes) as Rating, v.Category from VenueReview r
    Inner Join Venue v on v.VenName = r.Ven_Name
    Group By v.Category
    Order By Rating;
     """, connection)

print(avgRating.head(10))

   Rating    Category
0     0.0         Bar
1     0.0       Hotel
2     0.0        Park
3     0.0  Restaurant
4     0.0     Stadium


In [16]:
age = pd.read_sql("""
SELECT u.Username, 
    ABS(CAST((strftime('%Y', r.VRCreatedAt) - strftime('%Y', u.DOB)) - 
    (strftime('%m%d', r.VRCreatedAt) < strftime('%m%d', u.DOB)) AS INT)) AS age
FROM UserProfile u
JOIN VenueReview r ON u.Username = r.Username
GROUP BY u.Username
ORDER BY age;


     """, connection)

print(age.head(10))

     Username  age
0  002d4PaR86    0
1  00eoAY0q71    0
2  01FdwpMw21    0
3  01Fx3swD06    0
4  01G4TWv290    0
5  01NNePIt10    0
6  01bQNXQ302    0
7  01jQWV7S40    0
8  01lbmglp80    0
9  01oVEjQ696    0


In [19]:
prodRating = pd.read_sql("""
SELECT AVG(PRlikes) AS Rating, PicorVidID 
FROM ProductReview
GROUP BY PicorVidID
ORDER BY Rating;
     """, connection)

print(prodRating.head(10))

   Rating  PicorVidID
0     0.0  0003kJrYOj
1     0.0  0007MSUqq6
2     0.0  000KMjAvKV
3     0.0  000dApCqDB
4     0.0  000fKMu3ws
5     0.0  000kSW4CLP
6     0.0  000lifg6qJ
7     0.0  000vmUo3rR
8     0.0  001Flfs0qf
9     0.0  001HjvqqPC


In [22]:
prodInfo = pd.read_sql("""
Select productName, ProdCategory, price From Product 
where price > 200 AND price < 500
Group by productName
Order By price;
""", connection)

print(prodInfo.head(10))

  ProductName ProdCategory   Price
0    somebody      Outdoor  200.01
1      accept      Kitchen  200.22
2        full       Sports  200.43
3     station     Clothing  200.82
4     mission   Home Decor  200.87
5         hit      Kitchen  200.92
6        away       Sports  201.17
7         his      Kitchen  201.49
8         get     Bathroom  201.83
9       thing       Sports  202.03
